<center> <b>Geospatial Fellows Program</b> </center>


# High Accuracy Regions to Facilitate Road Traffic Metric Estimation from Camera Feeds: A Case in Central Ohio 
<i>Ningchuan Xiao, Yue Lin</i>

This Jupyter notebook demostrates the methods to promote the estimation of road traffic metrics using camera feeds. The proposed methods are applied to the traffic camera data collected in Central Ohio for accurate traffic density estimation.

## Notebook Outline
- [Problem Statement](#problem)

<a id='problem'></a>
## Problem Statement
The growing number of real-time camera feeds in urban areas have made it possible to provide high-quality traffic data for effective transportation management and control. However, making reliable traffic estimation has been a challenge due to the limitations of current vehicle detection techniques, as well as the various camera conditions such as height and resolution.

**The purpose of this paper is to develop a robust method that promotes the accuracy of traffic estimates derived from camera feeds**. A quadtree-based image segmentation algorithm is developed to extract the image regions where high vehicle detection accuracy can be achieved for each camera. These regions are also referred to as the high-accuracy identification regions (HAIR) in this paper. We further illustrate how proper use of the HAIR can help to improve the accuracy of traffic density estimates using the images from traffic cameras at different heights and resolutions in Central Ohio.